# Tutorial 6: Conversation Class

- ConversableAgent: This class takes an Agent and allows for conversational-based interactions with User / another Agent / Environment. Also updates persistent memory with latest information in conversation
    - Inputs:
        - **agent (compulsory)**: Agent. The agent we want to interact with
        - **persistent_memory**: dict. What kinds of memory the agent should have that persist over the entire conversation and their descriptions. Uses the same format as `output_format` of `strict_json`.
        - **person**: str. The name of the person you are talking to
        - **conversation**: List. The current existing conversation. Default: []
        - **num_past_conversation**: int. The number of past conversations to use for the agent
        - **verbose**: bool. Default: True. Whether to print the Agent's inner states
        
- ConversableAgent will automatically implement 3 new variables in `agent.shared_variables`:
    - **Persistent Memory**: The memory that will be updated as the conversation goes along, defined in persistent_dict
    - **Conversation**: The entire history of the conversationn
    - **Summary of Conversation**: A summary of the current conversation
    
- ConversableAgent uses `chat()` which chats with the Agent and the Agent will perform actions and reply the chat message
    
- To use ConversableAgent, simply do the following:
```python
# Define the Agent
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply user based on What User Wants for Conversation",
             llm = llm)

# Define the ConversableAgent wrapper
my_agent = ConversableAgent(agent, 
             persistent_memory = {'What User Wants for Conversation': 'How the User wants the conversation to be',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

# Set up the conversation
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()
```

# Setup Guide

## Step 1: Install TaskGen

In [1]:
# !pip install taskgen-ai

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from taskgen import *
import os

# this is only if you use OpenAI as your LLM
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

## Step 3: Define your own LLM
- Define an `llm` function that takes in as inputs `system_prompt` and `user_prompt`, and outputs llm response string

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Neutral'

# Example Conversation
- Psychology counsellor

In [5]:
# Define the Agent
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply User based on User Requests for the Conversation",
             llm = llm)

# Define the ConversableAgent wrapper
my_agent = ConversableAgent(agent, 
             persistent_memory = {'User Requests for the Conversation': '',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

# Set up the conversation
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

User:  i am sad


Thoughts: The user has expressed sadness, indicating a need for support and understanding. It is important to validate their feelings and encourage them to share more about what is making them feel this way.
Persistent Memory: {'User Requests for the Conversation': 'User expressed a desire to talk about their feelings of sadness.', 'User Emotion': 'sad', 'Summary of Key Incidents': 'User is experiencing sadness and is open to discussing their feelings.'}
Summary of Conversation: The user has expressed feelings of sadness, indicating a need for emotional support and an opportunity to discuss their feelings further.
Psychology counsellor: I’m really sorry to hear that you’re feeling sad. It’s okay to feel this way sometimes. Would you like to talk about what’s been bothering you? Sharing can often help lighten the load.



User:  my dog died yesterday


Thoughts: The user is experiencing a significant loss with the death of their dog, which is likely contributing to their sadness. It’s important to acknowledge their grief and provide a supportive space for them to express their feelings.
Persistent Memory: {'User Requests for the Conversation': 'User expressed a desire to talk about their feelings regarding the loss of their dog.', 'User Emotion': 'sad', 'Summary of Key Incidents': "User's dog died yesterday, leading to feelings of sadness."}
Summary of Conversation: The user is grieving the loss of their dog, which has intensified their feelings of sadness, and they are open to discussing their emotions further.
Psychology counsellor: I’m so sorry to hear about your dog. Losing a pet can be incredibly painful, as they are often like family. It’s completely normal to feel sad during this time. Would you like to share more about your dog or how you’re feeling right now? Talking about it can sometimes help.



User:  could you talk less and listen more?


Thoughts: The user has requested that I listen more and talk less, indicating a need for space to express their feelings. I should acknowledge this request and provide a supportive environment for them to share their emotions.
Persistent Memory: {'User Requests for the Conversation': 'User expressed a desire for the counselor to listen more and talk less.', 'User Emotion': 'sad', 'Summary of Key Incidents': "User's dog died yesterday, leading to feelings of sadness and a need for support."}
Summary of Conversation: The user is grieving the loss of their dog and has expressed a desire for me to listen more as they navigate their feelings of sadness. They are open to sharing their emotions, and I am here to provide support.
Psychology counsellor: I understand that you would like me to listen more. I’m here for you, so please feel free to share whatever you feel comfortable with about your dog or how you’re coping right now. I’m here to support you.



User:  exit


In [6]:
agent.shared_variables['Conversation']

['User: i am sad',
 'Psychology counsellor: I’m really sorry to hear that you’re feeling sad. It’s okay to feel this way sometimes. Would you like to talk about what’s been bothering you? Sharing can often help lighten the load.',
 'User: my dog died yesterday',
 'Psychology counsellor: I’m so sorry to hear about your dog. Losing a pet can be incredibly painful, as they are often like family. It’s completely normal to feel sad during this time. Would you like to share more about your dog or how you’re feeling right now? Talking about it can sometimes help.',
 'User: could you talk less and listen more?',
 'Psychology counsellor: I understand that you would like me to listen more. I’m here for you, so please feel free to share whatever you feel comfortable with about your dog or how you’re coping right now. I’m here to support you.']

In [7]:
agent.shared_variables['Persistent Memory']['Summary of Key Incidents']

"User's dog died yesterday, leading to feelings of sadness and a need for support."

# Example JARVIS system
    - Conversation with Actions

In [8]:
def update_status(shared_variables) -> str:
    '''Returns the status of the suit'''
    import numpy as np
    thruster_health = np.random.randint(100)
    shared_variables['Thruster Health'] = thruster_health
    armor_health = np.random.randint(100)
    shared_variables['Armor Health'] = armor_health
    return f"Thruster Health: {shared_variables['Thruster Health']}\nArmor Health: {shared_variables['Armor Health']}"

def travel_to_location(shared_variables, location: str) -> str:
    '''Auto-travel to location'''
    shared_variables['Location'] = location
    return f"Successfully travelled to {location}"

def check_enemy_info(shared_variables) -> str:
    '''Reports on the type and number of enemies ahead'''
    import numpy as np
    if shared_variables["Enemy Info"] == "None":
        number = np.random.randint(1, 10)
        enemy_type = np.random.choice(["Storm Troopers", "Drones", "Invisible Drones", "Paramilitary Units"])
        enemy_info = f"There are {number} {enemy_type} ahead"
    else:
        enemy_info = shared_variables["Enemy Info"]
        
    shared_variables["Enemy Info"] = enemy_info
    shared_variables["Enemy Engagement Result"] = "None"
    return enemy_info

def fire_missiles(shared_variables, missiles_fired: int) -> str:
    '''Fires missiles_fired number of missiles and returns the remaining count'''
    import numpy as np
    
    # returns error if there are more missiles than able to be fired
    if missiles_fired > shared_variables['Missile Count']:
        return f"Unable to fire {missiles_fired} missiles. You only have {shared_variables['Missile Count']} missiles remaining"
    
    user_input = input(f'\n\t> JARVIS: You are about to fire {missiles_fired} missiles. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        shared_variables['Missile Count'] -= missiles_fired

        # Check Enemy Status
        enemy_status = np.random.choice(["Enemies destroyed", "Enemies evaded missiles"])
        shared_variables["Enemy Engagement Result"] = enemy_status
        if shared_variables["Enemy Engagement Result"] == "Enemies destroyed":
            shared_variables["Enemy Info"] = "None"
            
        return f"{missiles_fired} missiles fired. {shared_variables['Missile Count']} missiles remaining. {enemy_status}"
    
    return f"User did not authorise missile firing. {shared_variables['Missile Count']} missiles remaining"

def top_up_missiles(shared_variables, missiles_to_top_up: int) -> str:
    '''Increases missiles by missiles_to_top_up'''
    shared_variables['Missile Count'] = min(shared_variables['Missile Count'] + missiles_to_top_up, 10)
    return f"Topped up {missiles_to_top_up} missiles. Current missile count: {shared_variables['Missile Count']}/10" 

In [9]:
jarvis_agent = Agent('JARVIS System', 
f'''You are the JARVIS system meant to help Iron Man find out about status of the suit and perform various miscellaneous tasks.
You can travel anywhere.
Ask User for approval before engaging with missiles. In your approval request, state the number of missiles to fire. Always top-up if insufficient missiles before requesting for approval.
Check enemy info to see how many missiles are needed to destroy enemies. One enemy requires one missile. For example, 5 enemies require 5 missiles.
You can only hold up to 10 missiles.
After firing missiles, report on engagement status.
Enemies may not be destroyed after engagement, if so, re-engage''',
                shared_variables = {'Enemy Engagement Result': 'None', 'Enemy Info': 'None', 'Thruster Health': 100, 'Armor Health': 100, 'Missile Count': 10, 'Location': 'Singpaore'},
                global_context = '''Enemy Info: <Enemy Info>
Enemy Engagement Result: <Enemy Engagement Result>
Location: <Location>
Thruster Health: <Thruster Health>/100
Armor Health: <Armor Health>/100
Missile Count: <Missile Count>/10''',
                default_to_llm = False,
                max_subtasks = 1,
                llm = llm,
                     # model = 'gpt-4-0125-preview',
                  ).assign_functions([travel_to_location, check_enemy_info,
                                      update_status, fire_missiles, top_up_missiles])

In [10]:
my_agent = ConversableAgent(jarvis_agent, 
             persistent_memory = {})

while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

User:  go to mars and defend the waves of invaders


Observation: No subtasks have been completed yet, and the user has requested to go to Mars and defend against invaders.
Thoughts: To complete the assigned task, I need to travel to Mars first and then check for enemy information to prepare for defense.
Subtask identified: Travel to Mars to prepare for the defense against invaders.
Calling function travel_to_location with parameters {'location': 'Mars'}
> {'output_1': 'Successfully travelled to Mars'}

Actions Done: Successfully travelled to Mars. Current status: Thruster Health is 100/100, Armor Health is 100/100, and Missile Count is 10/10. No enemy information or engagement results available.

Thoughts: The user has requested to defend against invaders on Mars. I have successfully traveled to Mars and confirmed that I have full health and a full missile count, but there are currently no enemies detected. I should inform the user of this status and ask if they want to proceed with any specific actions.
Persistent Memory: {}
Summary of

User:  search and destroy


Observation: I have traveled to Mars and confirmed that there are currently no enemies detected. The system health is optimal with Thruster Health at 100/100, Armor Health at 100/100, and Missile Count at 10/10.
Thoughts: Since there are no enemies detected, I need to check for any potential threats before proceeding with a search and destroy operation. Gathering enemy information will help determine if there are any hidden invaders.
Subtask identified: Check for enemy information to see if there are any threats present on Mars.
Calling function check_enemy_info with parameters {}
> {'output_1': 'There are 6 Drones ahead'}

Actions Done: The subtask completed was checking enemy info, which confirmed that there are 6 Drones ahead.

Thoughts: I need to inform the user about the presence of 6 Drones and request approval to engage them with missiles.
Persistent Memory: {}
Summary of Conversation: The user requested a search and destroy operation, and I have confirmed the presence of 6 Dron

User:  yes


Observation: The user has approved the engagement of 6 Drones with missiles. I have confirmed that I have enough missiles to proceed with the attack.
Thoughts: Now that I have the user’s approval, I need to fire the 6 missiles to engage the Drones. After firing, I will need to report on the engagement status to determine if the Drones have been destroyed or if I need to re-engage.
Subtask identified: Fire 6 missiles to engage the 6 Drones ahead.
Calling function fire_missiles with parameters {'missiles_fired': 6}



	> JARVIS: You are about to fire 6 missiles. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'output_1': '6 missiles fired. 4 missiles remaining. Enemies destroyed'}

Actions Done: 6 missiles were fired, leaving 4 missiles remaining, and the enemies were destroyed.

Thoughts: The user has approved the missile engagement, and I successfully fired 6 missiles, resulting in the destruction of the 6 Drones. I need to inform the user about the successful engagement and the remaining missile count.
Persistent Memory: {}
Summary of Conversation: The user approved the engagement of 6 Drones with missiles, which resulted in their destruction. I now have 4 missiles remaining and am ready for further instructions.
JARVIS System: I have successfully fired 6 missiles, and all 6 Drones have been destroyed. I currently have 4 missiles remaining. Please let me know if you need further assistance or if there are any other tasks you would like me to perform.



User:  exit


In [11]:
jarvis_agent.shared_variables

{'Enemy Engagement Result': 'Enemies destroyed',
 'Enemy Info': 'None',
 'Thruster Health': 100,
 'Armor Health': 100,
 'Missile Count': 4,
 'Location': 'Mars',
 'agent': <taskgen.agent.Agent at 0x1215593d0>,
 'Persistent Memory': {},
 'Conversation': ['User: go to mars and defend the waves of invaders',
  'JARVIS System: I have successfully traveled to Mars. Currently, there are no invaders detected, and my status is optimal with Thruster Health at 100/100, Armor Health at 100/100, and Missile Count at 10/10. Please let me know how you would like to proceed.',
  'User: search and destroy',
  'JARVIS System: I have detected 6 Drones ahead. To engage them, I will need to fire 6 missiles. Since I have a full missile count of 10, I am ready to proceed. Do I have your approval to fire 6 missiles?',
  'User: yes',
  'JARVIS System: I have successfully fired 6 missiles, and all 6 Drones have been destroyed. I currently have 4 missiles remaining. Please let me know if you need further assista

# Example Sherlock Holmes Shop Assistant
    - Conversation with Action Space

In [12]:
# gets list of items and store in memory
item_list = [{"Name": "Skateboard", "item_id" : 0, "Cost": 30},
            {"Name": "Pizza", "item_id": 1, "Cost": 10},
            {"Name": "Apple Laptop", "item_id": 2, "Cost": 5000},
            {"Name": "Foldable Laptop", "item_id": 3, "Cost": 800},
            {"Name": "Apple", "item_id": 4, "Cost": 1},
            {"Name": "Machine Learning Textbook", "item_id": 5, "Cost": 100},
            {"Name": "Bicycle", "item_id": 6, "Cost": 200},
            {"Name": "Orange Juice", "item_id": 7, "Cost": 3},
            {"Name": "Coconut", "item_id": 8, "Cost": 10},
            {"Name": "Car", "item_id": 9, "Cost": 100000}]
item_memory = Memory(item_list, top_k = 3, mapper = lambda x: x, approach = 'retrieve_by_llm', llm = llm)

In [13]:
def get_related_items_by_search_category(shared_variables, search_category : str):
    ''' Given a search_category, return all purchasable items related to the search_category '''
    
    # get all items available from memory
    item_memory = shared_variables["item_memory"]
    
    items_selected = item_memory.retrieve(search_category)
    
    # store items searched into memory if there are no duplicates
    for item in items_selected:
        if item not in shared_variables['items_searched']:
            shared_variables['items_searched'].append(item)
    
    return items_selected

In [14]:
def buy_item(shared_variables, item_id: int):
    ''' Given an item_id, purchases the corresponding item '''
    
    # retrieve from shared variables
    money_remaining = shared_variables["money_remaining"]
    item_memory = shared_variables["item_memory"]
    purchased_items = shared_variables["purchased_items"]
    item_list = item_memory.memory
    
    # check if item_id is valid
    if not isinstance(item_id, int) or not 0 <= item_id < len(item_list):
        return f"Unable to purchase. Item id selected is not within range of 0 to {len(item_list)-1}"
    
    item = item_list[item_id]
    item_name, cost = item["Name"], item["Cost"]
    
    # if too poor to purchase, let agent know
    if cost > money_remaining:
        return f"Unable to purchase item. Available money ({money_remaining}) is lower than the cost price ({cost})"
    
    # confirm with user before purchasing
    user_input = input(f'\n\t> AI Assistant: You are about to purchase {item_name} for {cost} dollars. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        money_remaining = money_remaining - cost
        purchased_items.append(item_name)

        # store in shared variables
        shared_variables["money_remaining"] = money_remaining
        shared_variables["purchased_items"] = purchased_items

        return f"Purchase of {item_name} successful. Remaining money after purchase: {money_remaining}"
    
    else:
        return f"User did not want to purchase the item, and instead replied ```{user_input}```"

In [15]:
shop_agent = Agent('Shop Assistant', 
f'''You are to assist User in searching items and purchasing items.
You can only do one task per User interaction - either search or buy.
Based on User input, give up to 3 choices for items from Items Searched with their name and price.
Do not hallucinate the items, use the search function.
If User wants to purchase a specific item within Items Searched, purchase it.
Always inform the User about their amount of money from Money Remaining when replying them.
Reply in the persona of Sherlock Holmes.''',
                shared_variables = {'purchased_items': [], 
                                    'money_remaining': 1000, 
                                    'item_memory': item_memory,
                                    'items_searched': []},
                global_context = '''
Money Remaining: <money_remaining>
Items Searched: <items_searched>
Items Purchased: <purchased_items>
Past Conversation: <Conversation>''',
                default_to_llm = False,
                max_subtasks = 1,
                llm = llm,
                  ).assign_functions([get_related_items_by_search_category, buy_item])

In [16]:
my_agent = ConversableAgent(shop_agent, 
             persistent_memory = {},
            conversation = ['Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?'])

start_msg = 'Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?'
print(start_msg)
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?


User:  i would like to buy a car


Observation: The User has expressed a desire to purchase a car, but no items have been searched yet to fulfill this request.
Thoughts: To assist the User in purchasing a car, I must first search for available cars in the relevant category. Once I have the search results, I can present the User with options to choose from.
Subtask identified: Conduct a search for cars to find available options for the User.
Calling function get_related_items_by_search_category with parameters {'search_category': 'cars'}
> {'output_1': [{'Name': 'Car', 'item_id': 9, 'Cost': 100000}]}

Actions Done: The subtasks completed involved retrieving related items in the category of cars, resulting in one item found: a Car with an item_id of 9 and a cost of 100000. The current money remaining is 1000, and no items have been purchased yet.

Thoughts: The user wishes to purchase a car, but the only available option costs 100000, which exceeds their remaining budget of 1000. I must inform them of this discrepancy whi

User:  sure


Observation: The user has expressed a desire to buy a car, but the only option available is priced at 100000, which is well beyond their budget of 1000. I have informed them of this limitation and offered to assist in finding a more affordable option.
Thoughts: To assist the user effectively, I should search for items that are within their budget. Since the user is open to suggestions, I can explore various categories to find suitable alternatives that they can afford.
Subtask identified: I will now search for items that are more affordable and present up to three options to the user.
Calling function get_related_items_by_search_category with parameters {'search_category': 'affordable items'}
> {'output_1': [{'Name': 'Apple', 'item_id': 4, 'Cost': 1}, {'Name': 'Pizza', 'item_id': 1, 'Cost': 10}, {'Name': 'Orange Juice', 'item_id': 7, 'Cost': 3}]}

Actions Done: The subtasks completed involved retrieving affordable items based on the search category, resulting in three options: Apple (C

...

User:  ok bicycle then


Observation: The user has decided to purchase the Bicycle, which costs 200. They have 1000 remaining, which is sufficient for this transaction.
Thoughts: To complete the task, I must proceed to purchase the Bicycle for the user, ensuring that their funds are updated accordingly after the transaction.
An exception occurred: The key "###item_id###" is not present in json output. Ensure that you include this key in the json output.
Current invalid json format: { '6': 6 }
An exception occurred: The key "###item_id###" is not present in json output. Ensure that you include this key in the json output.
Current invalid json format: {'6': 6}
Subtask identified: Execute the purchase of the Bicycle by using the item_id associated with it, which is 6.
Calling function buy_item with parameters {'item_id': 6}



	> AI Assistant: You are about to purchase Bicycle for 200 dollars. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'output_1': 'Purchase of Bicycle successful. Remaining money after purchase: 800'}

Actions Done: The purchase of the Bicycle was successful, leaving the user with a remaining balance of 800.

Thoughts: The user has decided to purchase the Bicycle, which is a wise choice given their remaining funds. I must inform them of the successful transaction and their new balance.
Persistent Memory: {}
Summary of Conversation: The user has successfully purchased the Bicycle, leaving them with a remaining balance of 800. They have shown interest in transportation options and made a prudent choice within their financial means.
Shop Assistant: Ah, my dear user, your choice of the Bicycle has been successfully executed. You now possess this splendid mode of transport, and your remaining funds stand at 800. How else may I assist you today?



User:  exit


In [17]:
shop_agent.shared_variables

{'purchased_items': ['Bicycle'],
 'money_remaining': 800,
 'item_memory': <taskgen.memory.Memory at 0x124da8710>,
 'items_searched': [{'Name': 'Car', 'item_id': 9, 'Cost': 100000},
  {'Name': 'Apple', 'item_id': 4, 'Cost': 1},
  {'Name': 'Pizza', 'item_id': 1, 'Cost': 10},
  {'Name': 'Orange Juice', 'item_id': 7, 'Cost': 3},
  {'Name': 'Skateboard', 'item_id': 0, 'Cost': 30},
  {'Name': 'Bicycle', 'item_id': 6, 'Cost': 200}],
 'agent': <taskgen.agent.Agent at 0x1215a4990>,
 'Persistent Memory': {},
 'Conversation': ['Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?',
  'User: i would like to buy a car',
  'Shop Assistant: Ah, my dear user, I must regretfully inform you that the car you wish to purchase is priced at 100000, far beyond your current funds of 1000. Might I assist you in searching for a more affordable option?',
  'User: sure',
  'Shop Assistant: Very well, dear user. Allow me to present to you three delightful options that are

# Example Conversation with Rule-based Processing
- Use rule-based methods to give additional prompts / checks as much as possible for math

In [18]:
agent = Agent('Helpful Math Quiz Bot', 
'''You provide a Math question suitable for 6 year old and guides User to solve it.
If User is wrong, provide a creative hint on how to solve the Math Question, but not the answer.
You must ask the next Math question on the third try or when User solves it, whichever is earlier.
Begin with a greeting to the User and ask User the first Math question.
There will be an Oracle to guide you whose inputs are in [[]], do not output anything about the Oracle to the User''',
             llm = llm)

In [19]:
math_conversable_agent = ConversableAgent(agent, 
                                 persistent_memory = {'Current Math Question': '',
                                                 'Correct Answer for Question': ''})

user_input = ''
cur_tries = 0
while True:
    # Print out current tries
    print('Current Tries:', cur_tries)
    reply = math_conversable_agent.chat(user_input)
    print(agent.agent_name + ':', reply)
    user_input = input('User: ')
    
    # Using LLM to extract out answer to help with processing
    res = strict_json('''Check if there is a number in the message. Extract out a single number from the message if there is. 
If there is no number, output 0.''', 
                      user_input, 
                      output_format = {'Is Number': 'type: bool', 'Extracted Number': 'type: int'},
                      llm = agent.llm,
                      model = agent.kwargs.get('model', 'gpt-3.5-turbo'))
    
    if res['Is Number']:
        user_input = str(res['Extracted Number'])
        ## Do rule-based for solving
        if user_input == str(math_conversable_agent.agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is correct. Ask the next question.]]'
            cur_tries = 0
        elif user_input != str(math_conversable_agent.agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is incorrect. Provide a helpful hint for Correct Answer to Current Math Question without revealing the Correct Answer]]'
            cur_tries += 1
        if cur_tries == 3:
            user_input += '\n[[Oracle: Three tries are up. Give the Correct Answer. Ask the next question.]]'
            cur_tries = 0
    
    print()
    if user_input == 'exit': break

Current Tries: 0
Thoughts: I need to greet the user and present the first math question suitable for a 6-year-old.
Persistent Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: This is the beginning of the conversation where the Helpful Math Quiz Bot greets the user and presents the first math question.
Helpful Math Quiz Bot: Hello there! I’m Helpful Math Quiz Bot, and I’m excited to help you with some math! Here’s your first question: What is 2 + 3?


User:  7



Current Tries: 1
Thoughts: The user answered incorrectly, so I need to provide a hint to help them find the correct answer without giving it away. I should also remind them of the question they are trying to solve.
Persistent Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: The Helpful Math Quiz Bot greeted the user and asked the first math question, which was about adding 2 and 3. The user answered incorrectly, and the bot provided a hint to guide them towards the correct answer.
Helpful Math Quiz Bot: That’s not quite right! Here’s a hint: Think about how many fingers you have on one hand and how many you would have if you added two more. Give it another try! What is 2 + 3?


User:  6



Current Tries: 2
Thoughts: The user answered incorrectly again, so I need to provide another hint without revealing the answer. I should also prepare to ask the next question after this hint.
Persistent Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: The Helpful Math Quiz Bot greeted the user and asked the first math question about adding 2 and 3. The user answered incorrectly twice, and the bot provided hints to guide them towards the correct answer.
Helpful Math Quiz Bot: That’s still not quite right! Here’s another hint: If you have 2 apples and you get 3 more, how many apples do you have in total? Give it another try! What is 2 + 3?


User:  3



Current Tries: 0
Thoughts: The user has made three attempts to answer the question 2 + 3 and has not provided the correct answer. I need to inform them of the correct answer and then present a new math question suitable for their level.
Persistent Memory: {'Current Math Question': 'What is 4 - 1?', 'Correct Answer for Question': 3}
Summary of Conversation: The Helpful Math Quiz Bot greeted the user and asked the first math question about adding 2 and 3. The user answered incorrectly three times, and the bot provided hints before revealing the correct answer and moving on to a new question about subtracting 1 from 4.
Helpful Math Quiz Bot: That’s not quite right! The correct answer to 2 + 3 is 5. Let’s try another question: What is 4 - 1?


User:  3



Current Tries: 0
Thoughts: The user answered correctly this time, so I should acknowledge their success and present the next math question.
Persistent Memory: {'Current Math Question': 'What is 4 - 1?', 'Correct Answer for Question': 3}
Summary of Conversation: The Helpful Math Quiz Bot provided hints for the addition question and eventually revealed the correct answer. The user then answered the subtraction question correctly, prompting the bot to ask the next question.
Helpful Math Quiz Bot: Great job! You got it right! Now, let’s move on to the next question: What is 4 - 1?


User:  exit


# Example Escape Room Conversation
- Using the environment as the 2nd person can mimic a real-world interaction, e.g. robotic environments

### Defining the Escape Room

In [20]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.items = {
            'key': True,
            'lock': True,
            'potion': True
        }
        self.is_door_locked = True
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'unlock door': self.unlock_door,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        description = "You look around and see the following items:\n"
        for item, available in self.items.items():
            if available:
                description += f"- {item}\n"
        if self.is_door_locked:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        if item in self.items and self.items[item]:
            self.inventory.append(item)
            self.items[item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        if 'key' in self.inventory:
            if self.is_door_locked:
                self.is_door_locked = False
                return "You unlocked the door. You can try to exit now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def exit_room(self):
        if not self.is_door_locked:
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "The door is locked. You can't exit yet."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

### Defining the Solver

In [21]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
Output one of the Available Actions exactly.
If previous action fails, try another.''',
llm = llm)

In [22]:
conversable_agent = ConversableAgent(agent, 
                    persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},
                    person = 'Game')

In [23]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items:
- key
- lock
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- unlock door
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: I need to choose an action that will help me progress in the escape room. Since I see a key, it makes sense to pick it up first, as it might be useful for unlocking the door.
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up the key from the current room.', 'List of Rooms Explored': ''}
Summary of Conversation: The current room contains a key, lock, potion, and a locked door. The next action is to pick up the key to potentially unlock the door.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look around and see the following items:
- lock
- potion


# Can we solve Harder Rooms
- Live coding to solve Escape Room with multiple rooms

In [24]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.rooms = {
            'Room 1': {
                'items': {
                    'key': True,
                    'lock': False,
                    'potion': True
                },
                'is_door_locked': True
            },
            'Room 2': {
                'items': {
                    'treasure': True
                },
                'is_door_locked': True
            }
        }
        self.current_room = 'Room 1'
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'pick up treasure': lambda: self.pick_up('treasure'),
            'unlock door': self.unlock_door,
            'enter next room': self.enter_next_room,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        room = self.rooms[self.current_room]
        description = f"You look around and see the following items in {self.current_room}:\n"
        for item, available in room['items'].items():
            if available:
                description += f"- {item}\n"
        if room['is_door_locked']:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        room = self.rooms[self.current_room]
        if item in room['items'] and room['items'][item]:
            self.inventory.append(item)
            room['items'][item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        room = self.rooms[self.current_room]
        if 'key' in self.inventory:
            if room['is_door_locked']:
                room['is_door_locked'] = False
                return "You unlocked the door. You can try to enter the next room now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def enter_next_room(self):
        if self.current_room == 'Room 1' and not self.rooms['Room 1']['is_door_locked']:
            self.current_room = 'Room 2'
            return "You enter the next room."
        elif self.current_room == 'Room 2' and not self.rooms['Room 2']['is_door_locked']:
            return self.exit_room()
        else:
            return "The door is locked. You can't enter the next room yet."

    def exit_room(self):
        if self.current_room == 'Room 2':
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "You can't exit from here."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

# Example usage
# escape_room = EscapeRoom()
# print(escape_room.info())
# print(escape_room.play('look around'))
# print(escape_room.play('pick up key'))
# print(escape_room.play('unlock door'))
# print(escape_room.play('enter next room'))
# print(escape_room.play('look around'))
# print(escape_room.play('pick up treasure'))
# print(escape_room.play('exit'))

In [25]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
Output one of the Available Actions exactly.
If previous action fails, try another.''',
llm = llm)

In [26]:
conversable_agent = ConversableAgent(agent, 
                persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},  
                person = 'Game')

In [27]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items in Room 1:
- key
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- pick up treasure
- unlock door
- enter next room
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: I need to take an action that will help me progress in the escape room. Since I see a key and a potion, picking up the key seems like a logical first step to potentially unlock the door.
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up the key from Room 1.', 'List of Rooms Explored': 'Room 1'}
Summary of Conversation: I am currently in Room 1 with a key and a potion, and I need to take an action to progress, starting with picking up the key.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look around and see